In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Animal Shelter python file
from animal_shelter import AnimalShelter

#===========================================================================
#===========================================================================
###########################
# Data Manipulation / Model
###########################

# MongoDB login credentials
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'],inplace=True)

#----------------------------------------------------------------------------
# Record numbers for table rows
df['rec_num'] = range(1, len(df) + 1)

# Rescue categories with relevant breeds
rescue_breeds = {
    "Water Rescue": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
    "Mountain or Wilderness Rescue": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
    "Disaster or Individual Tracking": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
}

#===========================================================================
#===========================================================================
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Dashboard Layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Horizontal title & logo with SNHU link
    html.Div([
        html.A([
            html.Img(src='/assets/grazioso_logo.png', style={'height': '80px', 'marginRight': '20px'})
        ], href='https://www.snhu.edu', target='_blank'),
        html.H1('Austin Animal Center Outcomes Dashboard', style={'marginTop': '15px'})
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'}),


    html.Div("Unique ID: Jaden Knutson", style={'textAlign': 'right', 'fontSize': '14px', 'color': 'gray'}),

    #----------------------------------------------------------------------------  
    # Filters for animals rescue types
    html.Label("Select Rescue Type"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'inline-block', 'margin-right': '15px', 'fontWeight': 'bold'}
    ),
    
    #----------------------------------------------------------------------------
    # Interactive DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": "Record #", "id": "rec_num", "selectable": True},
            {"name": "Age Upon Outcome", "id": "age_upon_outcome", "selectable": True},
            {"name": "Animal ID", "id": "animal_id", "selectable": True},
            {"name": "Animal Type", "id": "animal_type", "selectable": True},
            {"name": "Breed", "id": "breed", "selectable": True},
            {"name": "Color", "id": "color", "selectable": True},
            {"name": "Date of Birth", "id": "date_of_birth", "selectable": True},
            #{"name": "Datetime", "id": "datetime", "selectable": True},
            #{"name": "Month/Year", "id": "monthyear", "selectable": True},
            {"name": "Name", "id": "name", "selectable": True},
            {"name": "Outcome Subtype", "id": "outcome_subtype", "selectable": True},
            {"name": "Outcome Type", "id": "outcome_type", "selectable": True},
            {"name": "Sex Upon Outcome", "id": "sex_upon_outcome", "selectable": True},
            {"name": "Latitude", "id": df.columns[13], "selectable": True},
            {"name": "Longitude", "id": df.columns[14], "selectable": True},
        ],
        #----------------------------------------------------------------------------
        # Display Settings
        data=df.to_dict('records'),
        page_size=10,
        
        # Sorting and Filtering
        sort_action='native',
        filter_action='native',
        filter_options={'case': 'insensitive'},
        
        # Row Selection
        row_selectable='single',
        selected_rows=[0],

        # Column Selection and Styling
        column_selectable='single',
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        
        # Header Styling
        style_header={
            'paddingLeft': '10px',
            'paddingRight': '10px',
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'fontSize': '18px'
        },
    ),
     
    html.Br(),
    html.Hr(),
    
    #----------------------------------------------------------------------------
    # Horizontal layout for chart and map
    html.Div([
        dcc.Graph(id='outcome-chart', style={'width': '50%', 'display': 'inline-block'}),
        html.Div(id='map-id', style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'display': 'flex', 'flexWrap': 'nowrap', 'justifyContent': 'space-between'})

])

#===========================================================================
#===========================================================================
#############################################
# Interaction Between Components / Controller
#############################################
# Callback to highglight selected rows
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
#----------------------------------------------------------------------------
# Function to update styles
def update_styles(selected_columns, selected_rows):
    if selected_columns is None:
        selected_columns = []
    if selected_rows is None:
        selected_rows = []
    styles = []


    # Column highlight style
    styles.extend([
        {
            'if': {'column_id': i},
            'background_color': '#FFCCCC'  # light red
        } for i in selected_columns
    ])

    
    # Row highlight style
    styles.extend([
        {
            'if': {'row_index': i},
            'background_color': '#D2F3FF'  # light blue
        } for i in selected_rows
    ])

    return styles

#===========================================================================
#===========================================================================
# Callback updates map based on table selections
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
#----------------------------------------------------------------------------
# Function to update map
def update_map(viewData, index):    

    # Check if viewData available
    if not viewData or len(viewData) == 0:
        return [html.P("No data available.")]

    # Convert viewData to DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Default to first row if non selected
    if not index or index[0] >= len(dff):
        row = 0  
    else:
        row = index[0]

    # Ensure sufficient data size
    if dff.shape[1] < 15:
        return [html.P("Insufficient data to display map.")]

    # Generate map
    try:
        return [
            dl.Map(style={'width': '800px', 'height': '500px'},
                
                # Map Centering
                #center=[30.75, -97.48], zoom=9, children=[
                center=[dff.iloc[row, 13], dff.iloc[row, 14]], zoom=9, children=[
                    
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                        children=[
                            dl.Popup([
                                html.H4(f"Name: {dff.iloc[row, 9]}"),
                                html.P(f"Breed: {dff.iloc[row, 4]}")
                            ])
                        ])
            ])
        ]
    except Exception as e:
        return [html.P(f"Map error: {str(e)}")]

#===========================================================================
#===========================================================================    
@app.callback(
    Output('datatable-id', 'data'),
    Output('outcome-chart', 'figure'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_value):
    if filter_value == "Reset":
        dff = pd.DataFrame.from_records(shelter.read({}))
    else:
        query = {}
        if filter_value == "Water Rescue":
            query = {
                "breed": {"$in": rescue_breeds["Water Rescue"]},
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
                "sex_upon_outcome": "Intact Female"
            }
        elif filter_value == "Mountain or Wilderness Rescue":
            query = {
                "breed": {"$in": rescue_breeds["Mountain or Wilderness Rescue"]},
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
                "sex_upon_outcome": "Intact Male"
            }
        elif filter_value == "Disaster or Individual Tracking":
            query = {
                "breed": {"$in": rescue_breeds["Disaster or Individual Tracking"]},
                "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
                "sex_upon_outcome": "Intact Male"
            }

        dff = pd.DataFrame.from_records(shelter.read(query))

    if not dff.empty and '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    dff['rec_num'] = range(1, len(dff) + 1)

    fig = px.pie(dff, names="outcome_type", title="Outcome Type Distribution", hole=0.4)

    return dff.to_dict('records'), fig



#===========================================================================
#===========================================================================
# Run the Application
app.run_server(debug=True)


Dash app running on http://127.0.0.1:10374/
